<a href="https://colab.research.google.com/github/DeJezuz/AI-Future-Directions-Assignment/blob/main/models/recyclables_int8_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()  # Select your recyclables.zip file when prompted

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import zipfile
import os

# Unzip the uploaded file
with zipfile.ZipFile('recyclables.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/recyclables') # Extract to a directory named 'recyclables'

img_size = (160, 160)
batch_size = 32

# Adjust the directory path to account for the nested folder created by unzipping
base_dir = '/content/recyclables/recyclables'

# Load all images into train_ds and val_ds, as validation_split is problematic for 4 images
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    seed=42, # Keep seed for reproducibility if needed for other operations
    image_size=img_size, batch_size=batch_size
)
# For the purpose of representative dataset for quantization, val_ds can be the same as train_ds given the tiny dataset size.
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    base_dir,
    seed=42,
    image_size=img_size, batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)

### Define a simple CNN model

In [ ]:
model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(img_size[0], img_size[1], 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

model.summary()

### Compile the model

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

### Train the model (for demonstration purposes with very few images)

In [ ]:
epochs = 5 # Small number of epochs due to very limited data
history = model.fit(
    train_ds,
    epochs=epochs
)

In [ ]:
# Load a lightweight pretrained model
base_model = keras.applications.MobileNetV2(
    input_shape=img_size + (3,),  # (160, 160, 3)
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze the base model

# Add classification layers
inputs = keras.Input(shape=img_size + (3,))
x = keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# Compile and train
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_ds, epochs=5)

In [ ]:
# Create a representative dataset for quantization
def representative_data_gen():
    # Ensure val_ds is defined and contains data for this to work
    for images, _ in val_ds.take(50):
        yield [tf.cast(images, tf.float32)]

# Set up the converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert the model
tflite_model = converter.convert()

# Save the model to a file
with open("recyclables_int8.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ File saved: recyclables_int8.tflite")